### **Normalizar los datos con MinMaxScaler**

In [1]:
import pandas as pd

train = pd.read_csv('Z:\\Descargas\\composite\\deepl\\train_correct.csv')
train.shape

(4165824, 29)

In [2]:
train

,Flow rate,Zone2_Pressure,Zone3_Pressure,Zone4_Pressure,Zone5_Pressure,Zone6_Pressure,Zone7_Pressure,Zone9_Pressure,Zone10_Pressure,Zone11_Pressure,...,Zone21_Pressure,Zone22_Pressure,Zone23_Pressure,Zone24_Pressure,Zone26_Pressure,Zone27_Pressure,Zone28_Pressure,Zone29_Pressure,Zone30_Pressure,Zone31_Pressure
0,6.923975e-07,1852.592801,1807.574352,1663.689589,1520.194255,1356.602656,1129.024506,3167.188078,2310.766665,1919.164035,...,1524.034071,1349.307342,1212.632523,1131.491383,1857.934232,1806.640343,1664.708868,1503.278248,1334.735654,1158.354777
1,7.256684e-07,-657.722361,-640.950569,-591.066470,-538.044223,-481.535585,-400.869362,-1129.496306,-821.833946,-681.737573,...,-539.864771,-483.457895,-437.543667,-413.988056,-667.035535,-643.167738,-587.510925,-532.026228,-478.464222,-423.805870
2,8.672688e-07,373.215764,362.596499,334.567299,302.679773,271.303270,225.386404,645.770678,467.579185,386.534449,...,303.423444,274.213249,249.080758,238.612687,383.054509,365.135526,329.322693,298.208273,270.948979,243.835287
3,8.039416e-07,-253.402782,-245.248996,-226.309480,-203.210921,-182.296629,-150.926529,-442.452366,-318.469479,-262.113086,...,-203.336115,-185.252174,-168.595588,-163.436995,-263.300279,-247.750848,-220.123902,-199.030597,-182.523080,-166.563044
4,1.245084e-06,188.619684,181.779330,167.734162,149.347932,134.033797,110.482734,332.639715,237.872075,194.838756,...,149.084297,136.862630,124.659989,122.282727,198.460292,184.163864,160.918550,145.155671,134.329266,124.209637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4165819,-1.746591e-08,35955.612141,37939.488881,37792.662133,36297.031630,34146.040523,29654.069217,55982.155370,43326.320428,41695.880168,...,42638.271620,37217.694751,31514.850674,27258.475090,44615.652582,52938.894281,62304.188933,51008.897324,39466.403006,29445.131862
4165820,1.893676e-08,35701.554076,37752.743894,37705.329744,36270.559184,34150.946906,29682.122591,55392.864619,42995.065646,41493.846284,...,42585.062890,37259.048863,31675.257474,27523.486760,44256.635726,52509.806410,61797.176216,50777.683166,39494.125522,29728.497623
4165821,-2.107540e-08,36407.886750,38543.670036,38566.279727,37176.904105,35077.417728,30582.347823,56346.886115,43813.631497,42324.825752,...,43620.922063,38445.535720,33042.227052,29038.923383,45087.742526,53486.431560,62965.683227,52010.841984,40870.855263,31372.272055
4165822,2.457438e-08,35888.246224,38083.068680,38219.435137,36916.588154,34871.401560,30436.492974,55328.268619,43157.768587,41820.730735,...,43284.374571,38271.545717,33049.898785,29197.455514,44391.945314,52661.724056,61997.332143,51433.002679,40680.004355,31563.161216


In [3]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(train)
train = scaler.transform(train)
train.shape

(4165824, 29)

### **Definir modelo CNN-AutoEncoder**

In [4]:
# reshape input into [samples, timesteps, features]

n_samples = 5007
n_steps = 832
n_features = train.shape[1]
train = train.reshape((n_samples, n_steps, n_features))

train.shape

(5007, 832, 29)

In [10]:
from tensorflow.keras import models, layers

model = models.Sequential([
    # encoder
    layers.Conv1D(32, 3, activation='relu', input_shape=(train.shape[1], train.shape[2])),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(16, 3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(8, 3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    # decoder
    #layers.Conv1D(8, 3, activation='relu'),
    layers.UpSampling1D(2),
    layers.Conv1D(16, 3, activation='relu'),
    layers.UpSampling1D(3),
    layers.Conv1D(32, 3, activation='relu'),
    layers.UpSampling1D(2)
])

model.compile(optimizer='rmsprop', loss='mae')
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_18 (Conv1D)           (None, 830, 32)           2816      
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 415, 32)           0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 413, 16)           1552      
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 206, 16)           0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 204, 8)            392       
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 102, 8)            0         
_________________________________________________________________
up_sampling1d_8 (UpSampling1 (None, 204, 8)           